# Handling the data processing with family_main.ipynb

In [3]:
import os
import csv
import numpy as np
import pandas as pd

## Loading the data

In [110]:
csv_file_path = './data/family_data_raw.csv'

def load_csv(csv_file_path):
    ori_data = []
    with open(csv_file_path, mode='r', newline='') as file:
        reader = csv.reader(file)
        header = next(reader)  
        for row in reader:
            ori_data.append(row)

    df = pd.DataFrame(ori_data, columns=header)
    print(header)
    return df

df = load_csv(csv_file_path)
print(df.shape)

['Family', 'Member', 'Income', 'Spend']
(279, 4)


## Check the data quality
- Which is also Q4 required

In [111]:
print("DataFrame Information：")
print(df.info())

missing_data = df.isnull().sum()
missing_data

DataFrame Information：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Family  279 non-null    object
 1   Member  279 non-null    object
 2   Income  279 non-null    object
 3   Spend   279 non-null    object
dtypes: object(4)
memory usage: 8.8+ KB
None


Family    0
Member    0
Income    0
Spend     0
dtype: int64

In [113]:
# change the data into numeric for further usage
df['Income'] = pd.to_numeric(df['Income'], errors='coerce')
df['Spend'] = pd.to_numeric(df['Spend'], errors='coerce')

potential_errors = {
    "Negative_Income": df['Income'].min() < 0,
    "Negative_Spend": df['Spend'].min() < 0,
    "Missing_Data": df.isnull().any().any(),
    "Duplicate_Rows": df.duplicated().any()
}

potential_errors

{'Negative_Income': False,
 'Negative_Spend': False,
 'Missing_Data': False,
 'Duplicate_Rows': False}

In [114]:
df.head(5)

,Family,Member,Income,Spend
0,family1,Adult1,2376330,1119433
1,family1,Adult2,130268,37337
2,family1,Adult3,2254489,972327
3,family2,Adult1,2292355,649806
4,family2,Adult2,298167,100723


## Data preprocessing
- get the amound of the adult and child by adding column is_adult and is_child

In [115]:
df['Is_Adult'] = df['Member'].str.contains('Adult')
df['Is_Child'] = df['Member'].str.contains('Child')

df.head(5)

,Family,Member,Income,Spend,Is_Adult,Is_Child
0,family1,Adult1,2376330,1119433,True,False
1,family1,Adult2,130268,37337,True,False
2,family1,Adult3,2254489,972327,True,False
3,family2,Adult1,2292355,649806,True,False
4,family2,Adult2,298167,100723,True,False


In [68]:
# Group by family and calculate the statistics
family_stats = df.groupby('Family').agg(
    Member_Count=('Member', 'count'),
    Adult_Count=('Is_Adult', 'sum'),
    Child_Count=('Is_Child', 'sum'),
    Total_Income=('Income', 'sum'),
    Total_Spend=('Spend', 'sum')
).reset_index()

family_stats['Balance'] = family_stats['Total_Income'] - family_stats['Total_Spend']

# Convert the aggregated data to a dictionary format
family_stats_dict = family_stats.to_dict(orient='records')

In [87]:
family_stats_dict[0]

{'Family': 'family1',
 'Member_Count': 3,
 'Adult_Count': 3,
 'Child_Count': 0,
 'Total_Income': 4761087,
 'Total_Spend': 2129097,
 'Balance': 2631990}

## Problem Sets

- Q1 income problem

In [81]:
# Q1: 最高和最低年收入的家庭
highest_income_family = family_stats.loc[family_stats['Total_Income'].idxmax()]
lowest_income_family = family_stats.loc[family_stats['Total_Income'].idxmin()]

print(f"highest_income_family:\n{highest_income_family}")
print()
print(f"lowest_income_family:\n{lowest_income_family}")


highest_income_family:
Family          family6
Member_Count          5
Adult_Count           3
Child_Count           2
Total_Income    7804425
Total_Spend     2879221
Balance         4925204
Name: 56, dtype: object

lowest_income_family:
Family          family94
Member_Count           2
Adult_Count            1
Child_Count            1
Total_Income       46790
Total_Spend        30029
Balance            16761
Name: 94, dtype: object


- Q2: get deficit family by balance

In [84]:
# Q2: 年收入不足以覆盖支出的家庭及最大赤字
deficit_families = family_stats[family_stats['Balance'] < 0]
# max_deficit_family = deficit_families.loc[deficit_families['Balance'].idxmin()]
deficit_families

,Family,Member_Count,Adult_Count,Child_Count,Total_Income,Total_Spend,Balance


In [116]:
# double check 
for family in family_stats_dict:
    print(family['Balance'])
    if family['Balance'] < 0:
        print(f"{family['Family']} is deflicit")

2631990
1114981
773232
1767369
2438761
3725526
737428
93935
4637718
658493
3853874
3999720
2049463
1882237
1479632
1934310
1007551
2255039
232322
2693993
309806
666218
2523623
1494096
2114286
4516639
1570887
239964
1525898
1757668
434964
2213659
1299722
3182061
1767425
1700053
1344629
1244449
4168802
2199511
1605549
554209
3713084
123925
75863
926852
2858277
2788729
891975
3362599
819522
1906413
1391115
488997
329905
966411
4925204
1353020
889825
4338093
3469139
584858
797198
1659233
340588
39239
2906447
1516887
3374652
570284
441157
3876118
350551
2361148
3865964
2116581
128982
384992
1845569
521581
999930
485228
3314481
46290
1549274
1486152
744772
1136974
955266
2537824
1804735
1927620
2089264
1219583
16761
3710586
189108
1889100
1986654
1333572


- Q3: get single-parent and childless family

In [117]:
single_parent_families = family_stats[(family_stats['Adult_Count'] == 1) & (family_stats['Child_Count'] > 0)]
single_parent_families_selected = single_parent_families[['Family', 'Adult_Count', 'Child_Count']]

print(f"There are {single_parent_families_selected.shape[0]} of single parent families\n")
print("Single parent families:")
single_parent_families_selected

There are 23 of single parent families

Single parent families:


,Family,Adult_Count,Child_Count
6,family14,1,2
7,family15,1,2
14,family21,1,2
15,family22,1,1
20,family27,1,1
27,family33,1,2
31,family37,1,1
40,family45,1,1
43,family48,1,1
50,family54,1,1


In [118]:
childless_families = family_stats[family_stats['Child_Count'] == 0]
childless_families_selected = childless_families[['Family', 'Adult_Count', 'Child_Count']]

print(f"There are {childless_families_selected.shape[0]} of childless families\n")
print(f"Childless families:")
childless_families_selected

There are 35 of childless families

Childless families:


,Family,Adult_Count,Child_Count
0,family1,3,0
2,family100,2,0
4,family12,3,0
5,family13,2,0
9,family17,2,0
17,family24,2,0
18,family25,1,0
23,family3,1,0
26,family32,1,0
30,family36,2,0
